In [1]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 

os.listdir('/kaggle/input/')

['edited-coco-dataset']

In [2]:
import yaml

yaml_path = "/kaggle/input/edited-coco-dataset/data.yaml"

# Load YAML
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

# ✅ Update train and val
data["train"] = "/kaggle/input/edited-coco-dataset/outputs/filtered_dataset/train/images"
data["val"] = "/kaggle/input/edited-coco-dataset/outputs/filtered_dataset/val/images"

# Save updated YAML to a writeable directory
new_yaml_path = "/kaggle/working/data_balanced.yaml"

with open(new_yaml_path, "w") as f:
    yaml.safe_dump(data, f)

print("Updated data.yaml saved to:", new_yaml_path)

Updated data.yaml saved to: /kaggle/working/data_balanced.yaml


In [4]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 93.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from ultralytics import YOLO
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from ultralytics import YOLO

# Load your pretrained model
model = YOLO("/kaggle/input/edited-coco-dataset/yolo8_coco24_runs/yolo8_coco24_highacc/weights/best.pt")

# Fine-tune the model
model.train(
    data='/kaggle/working/data_balanced.yaml',
    epochs=10,
    batch=16,
    imgsz=640,
    workers=2,  # low for Kaggle
    
    optimizer="AdamW",
    lr0=0.0005,
    cos_lr=True,
    augment=True,
    dropout=0.2,
    patience=2,
    
    project="yolo8_coco24_runs",
    name="yolo8_coco24_finetuned"
)


Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data_balanced.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/edited-coco-dataset/yolo8_coco24_runs/yolo8_coco24_highacc/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8_coco24_finetuned, nbs=64, nms=False, opset=None,

In [ ]:
from ultralytics import YOLO

# Load last trained weights
model = YOLO("yolo8_coco24_runs/yolo8_coco24_highacc/weights/last.pt")

# Continue training for more epochs
model.train(
    data='/kaggle/working/data_balanced.yaml',
    epochs=3,     # ← add 10 more epochs
    batch=16,
    imgsz=640,
    workers=2,
    optimizer="AdamW",
    lr0=0.0005,
    cos_lr=True,
    augment=True,
    dropout=0.2,
    patience=2,
    project="yolo8_coco24_runs",
    name="yolo8_coco24_highacc"
)
